#### Dinossauro

In [7]:
import pygame
cor_dino = 255,255,255
altura_dino = 40
largura_dino = 20

# classe que faz todos os movimentos do dinossauro
class Dinosaur:
    def __init__ (self, surfaceHeight):    
        self.x = 60 # cordenada de spawn eixo X
        self.y = 0  # cordenada de spawn eixo Y
        self.yvelocity = 0 # Fica parado
        self.height = altura_dino # altura do objeto
        self.width = largura_dino # largura do objeto 
        self.surfaceHeight = surfaceHeight # altura em relaçao ao chao
        self.estado = False # em pé ou abaixado 
        self.altura_abaixado = 20 # metade da altura
        self.largura_abaixado = 30  # metade da largura
        
        
    def jump(self):
        if (self.y == 0): # <=> esta no chao ele pode pular
            self.yvelocity = 300 #velocidade no ar ou seja sobe
    def update(self, deltaTime): 
        self.yvelocity += -500*deltaTime # ele vai somar o 300 com -500 * deltatime, -500 simula a gravidade que multiplica com os frames da tela para suavizar
        self.y += self.yvelocity * deltaTime # atualiza a posiçao dele * deltatime que é o fps da tela
        if self.y < 0: # se ele ficar bugado no chao ou seja passar pra baixo dele ele vai atualizar para cima
            self.y = 0
            self.yvelocity = 0
    def pe(self):
        self.estado = False # 
        self.height = 40
        self.width = 20
    
    def abaixar(self):
        self.estado = True
        self.height = 20
        self.width = 25
        

    def draw(self,display):
        if self.estado == True:
            pygame.draw.rect(display,cor_dino,[self.x,self.surfaceHeight-self.y-self.height,self.width,self.height])
        else:
            pygame.draw.rect(display,cor_dino,[self.x,self.surfaceHeight-self.y-self.height,self.width,self.height])

    def check_collision(self,obstaculo):
        dino_rect = pygame.Rect(self.x, self.surfaceHeight- self.y -self.height, self.width,self.height)
        obstaculo_rect = pygame.Rect(obstaculo.x,obstaculo.y,obstaculo.width,obstaculo.height)

        return dino_rect.colliderect(obstaculo_rect)
        
            

#### Obstaculos

In [11]:
import pygame
class Obstaculos:
    def __init__(self,width,height,surface_width,surface_heigth, speed):
        self.width = width
        self.height = height
        self.x = surface_width #Eixo X dos objetos
        self.y = surface_heigth - height  # Altura do chao
        self.speed = speed
    def update(self,deltaTime):
        self.x -= self.speed * deltaTime
    def draw(self,display,color=(255,0,0)): 
        pygame.draw.rect(display,color,[self.x,self.y+10,self.width,self.height-10])
    def should_delete(self):
        return self.x+self.width <0 #remove quando o eixo x e a soma da altura for menor que zero ou seja saiu da tela.

class Obstaculo_passaro:
    def __init__(self,width,height,surface_width,surface_heigth, speed):
        self.width = width
        self.height = height
        self.x = surface_width #eixo X dos objetos
        self.y = surface_heigth - height - 25  # Altura do chao
        self.speed = speed
    def update(self,deltaTime):
        self.x -= self.speed * deltaTime
    def draw(self,display,color=(255,0,0)): 
        pygame.draw.rect(display,color,[self.x,self.y-5,self.width,self.height-10])
    def should_delete(self):
        return self.x+self.width <0 #remove quando o eixo x e a soma da altura for menor que zero ou seja saiu da tela.
        
        


#### Pontos

In [12]:

class Pontos:
    def __init__(self,canto_x = 40, canto_y = 40):
        self.width = canto_x
        self.height = canto_y
        self.pontos = 0
        self.tempo = pygame.time.get_ticks()
        self.intervalo = 1000
    def update(self):
        tempo_atual = pygame.time.get_ticks()
        if (tempo_atual - self.tempo) >= self.intervalo:
            self.pontos += 1
            self.tempo = tempo_atual
        
    def draw(self,display):
        font = pygame.font.Font(None,36)
        text = font.render(f"pontos:{self.pontos}", True,( 255,255,255))
        display.blit(text,(self.width,self.height))

#### Base

In [17]:
import pygame
import random

pygame.init()

v_inicial = 200
v_atual = v_inicial
aceleracao = 0.1

size = width, height = 640, 480
game_display = pygame.display.set_mode(size)

altura_chao = height - 100
preto = (0, 0, 0)
branco = (255, 255, 255)

dinosaur = Dinosaur(altura_chao)
lastFrame = pygame.time.get_ticks()

obstaculos = []
obstaculos_passaro = []
spawn_timer = 2
spawn_timer_passaro = 3.5  

running = True
game_over = False

pontos = Pontos()

while running:
    t = pygame.time.get_ticks()
    deltaTime = (t - lastFrame) / 1000
    lastFrame = t

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                dinosaur.jump()

            if event.key == pygame.K_DOWN:
                dinosaur.abaixar()

        if event.type == pygame.KEYUP:  
            if event.key == pygame.K_DOWN:
                dinosaur.pe()

    game_display.fill(preto)

    pygame.draw.rect(game_display, branco, [0, altura_chao, width, height - altura_chao])
    dinosaur.update(deltaTime)
    dinosaur.draw(game_display)

    
    spawn_timer -= deltaTime
    if spawn_timer <= 0:
        obstaculos.append(Obstaculos(20, 40, width, altura_chao, v_atual))
        spawn_timer = 2  

    
    spawn_timer_passaro -= deltaTime
    if spawn_timer_passaro <= 0:
        
        overlap = False
        for obstaculo in obstaculos:
            if abs(obstaculo.x - width) < 50:  
                overlap = True
                break

        if not overlap:
            obstaculos_passaro.append(Obstaculo_passaro(20, 50, width, altura_chao, v_atual))
        spawn_timer_passaro = 3.5  
    for k in obstaculos + obstaculos_passaro:
        k.speed = v_atual

    
    obstaculos = [i for i in obstaculos if not i.should_delete()]
    for i in obstaculos:
        i.update(deltaTime)
        i.draw(game_display, color=(255, 0, 0))  # Cactos em vermelho

    
    obstaculos_passaro = [j for j in obstaculos_passaro if not j.should_delete()]
    for j in obstaculos_passaro:
        j.update(deltaTime)
        j.draw(game_display, color=(0, 0, 255))  # Pássaros em azul

    
    for obstaculo in obstaculos:
        if dinosaur.check_collision(obstaculo):
            game_over = True

    
    for obstaculo_passaro in obstaculos_passaro:
        if dinosaur.check_collision(obstaculo_passaro):
            game_over = True

    v_max = 500
    v_atual = min(v_atual, v_max) 
    
    if game_over:
        print('Game Over! Pontuação:', pontos.pontos)
        v_atual = v_inicial
        obstaculos = [] 
        obstaculos_passaro = []  
        pontos.pontos = 0  
        dinosaur = Dinosaur(altura_chao)  
        game_over = False 

        spawn_timer = 2
        spawn_timer_passaro = 3.5

    pontos.update()
    pontos.draw(game_display)

    v_atual = v_inicial + (pontos.pontos*5)

    # coleta de dados para rede neural: 
    def dados(dinosaur, obstaculos, obstaculos_passaro, v_atual):
        todos_obstaculos = obstaculos + obstaculos_passaro
        if not todos_obstaculos:
            return [width, 0, v_atual, dinosaur.y, 0]  # Retorno padrão sem obstáculos
        
        obstaculo_mais_proximo = min(todos_obstaculos, key=lambda o: o.x)
        return [
            obstaculo_mais_proximo.x,          # Distância (X)
            obstaculo_mais_proximo.height,      # Altura do obstáculo
            v_atual,                            # Velocidade do jogo
            dinosaur.y,                         # Posição Y do dinossauro
            0 if dinosaur.estado else 1         # 1=abaixado, 0=em pé (CORRIGIDO: era 'pe')
        ]
        
            
    jogo = dados(dinosaur, obstaculos, obstaculos_passaro,v_atual)
    print(jogo)
    pygame.display.update()

pygame.quit()

[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0.0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0.0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0.0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0.0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0.0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0.0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0, 200, 0, 0]
[640, 0,